# Personalize with temporal evaluation on hold-out set

The notebook largely follows the basic notebook, with the additional tweak to hold-out 10% of "future" data for every user. Then, we set up an inference endpoint to bring recommendation and evaluate externally on the held-out data.

The other minor difference is that we predict views rather than ratings. We find it more interesting to demonstrate recommendation of unpopular (yet highly personalized) movies than to recommend popular movies that everyone would enjoy (without personalization needs).

In [1]:
import boto3, os
import json
import numpy as np
import pandas as pd
import time

In [2]:
os.environ['AWS_DEFAULT_REGION']="us-east-1"

In [3]:
bucket = "demo-temporal-holdout"           # replace with the name of your S3 bucket
filename = "DEMO-temporal-holdout.csv"

In [4]:
!aws s3 mb s3://{bucket}

make_bucket: demo-temporal-holdout


In [5]:
!curl -O https://s3-us-west-2.amazonaws.com/personalize-cli-json-models/personalize.json
!curl -O https://s3-us-west-2.amazonaws.com/personalize-cli-json-models/personalize-runtime.json
!aws configure add-model --service-model file://`pwd`/personalize.json --service-name personalize
!aws configure add-model --service-model file://`pwd`/personalize-runtime.json --service-name personalize-runtime

personalize = boto3.client(service_name='personalize', endpoint_url='https://personalize.us-east-1.amazonaws.com')
personalize_runtime = boto3.client(service_name='personalize-runtime', endpoint_url='https://personalize-runtime.us-east-1.amazonaws.com')

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  125k  100  125k    0     0   365k      0 --:--:-- --:--:-- --:--:--  365k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3106  100  3106    0     0  18378      0 --:--:-- --:--:-- --:--:-- 18378


# Download and process benchmark data

In [8]:
!curl -O http://files.grouplens.org/datasets/movielens/ml-1m.zip
!unzip -o ml-1m.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 5778k  100 5778k    0     0  11.5M      0 --:--:-- --:--:-- --:--:-- 11.5M
Archive:  ml-1m.zip
  inflating: ml-1m/movies.dat        
  inflating: ml-1m/ratings.dat       
  inflating: ml-1m/README            
  inflating: ml-1m/users.dat         


In [9]:
data = pd.read_csv('./ml-1m/ratings.dat', sep='::', names=['USER_ID','ITEM_ID','RATING','TIMESTAMP'])
pd.set_option('display.max_rows', 5)
data

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


,USER_ID,ITEM_ID,RATING,TIMESTAMP
0,1,1193,5,978300760
1,1,661,3,978302109
...,...,...,...,...
1000207,6040,1096,4,956715648
1000208,6040,1097,4,956715569


### Extract last 10% of interactions per user as hold-out tests

In [10]:
ranks = data.groupby('USER_ID').TIMESTAMP.rank(pct=True, method='first')

In [11]:
data = data.join((ranks>0.9).to_frame('holdout'))

In [12]:
holdout = data[data['holdout']].drop('holdout', axis=1)

In [13]:
data = data[~data['holdout']].drop('holdout', axis=1)

In [126]:
len(data['USER_ID'].unique())

6040

In [127]:
len(data['ITEM_ID'].unique())

3684

### Upload data

In [14]:
# data = data[data['RATING'] > 3.6]  # Use all data to predict view recommendations
data = data[['USER_ID', 'ITEM_ID', 'TIMESTAMP']] # select columns that match the columns in the schema below
data.to_csv(filename, index=False)
!head {filename}

boto3.Session().resource('s3').Bucket(bucket).Object(filename).upload_file(filename)

USER_ID,ITEM_ID,TIMESTAMP
1,1193,978300760
1,661,978302109
1,914,978301968
1,3408,978300275
1,1197,978302268
1,1287,978302039
1,2804,978300719
1,594,978302268
1,919,978301368


# Create Schema

In [15]:
schema = {
    "type": "record",
    "name": "Interactions",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        {
            "name": "USER_ID",
            "type": "string"
        },
        {
            "name": "ITEM_ID",
            "type": "string"
        },
        {
            "name": "TIMESTAMP",
            "type": "long"
        }
    ],
    "version": "1.0"
}

create_schema_response = personalize.create_schema(
    name = "DEMO-temporal-schema",
    schema = json.dumps(schema)
)

schema_arn = create_schema_response['schemaArn']
print(json.dumps(create_schema_response, indent=2))

{
  "schemaArn": "arn:aws:personalize:us-east-1:261294318658:schema/TEMP-temporal-schema",
  "ResponseMetadata": {
    "RequestId": "03f36318-df1c-4e86-8024-34c4ae970e3f",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Fri, 18 Jan 2019 18:35:55 GMT",
      "x-amzn-requestid": "03f36318-df1c-4e86-8024-34c4ae970e3f",
      "content-length": "86",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


# Create and Wait for Dataset Group

In [16]:
create_dataset_group_response = personalize.create_dataset_group(
    name = "DEMO-temporal-dataset-group"
)

dataset_group_arn = create_dataset_group_response['datasetGroupArn']
print(json.dumps(create_dataset_group_response, indent=2))

{
  "datasetGroupArn": "arn:aws:personalize:us-east-1:261294318658:dataset-group/TEMP-temporal-dataset-group",
  "ResponseMetadata": {
    "RequestId": "5869887c-6204-4247-b96d-38289292ea9b",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Fri, 18 Jan 2019 18:35:55 GMT",
      "x-amzn-requestid": "5869887c-6204-4247-b96d-38289292ea9b",
      "content-length": "106",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [18]:
status = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_group_response = personalize.describe_dataset_group(
        datasetGroupArn = dataset_group_arn
    )
    status = describe_dataset_group_response["datasetGroup"]["status"]
    print("DatasetGroup: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(20)

DatasetGroup: ACTIVE


In [19]:
dataset_type = "INTERACTIONS"
create_dataset_response = personalize.create_dataset(
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_arn,
    schemaArn = schema_arn
)

dataset_arn = create_dataset_response['datasetArn']
print(json.dumps(create_dataset_response, indent=2))

{
  "datasetArn": "arn:aws:personalize:us-east-1:261294318658:dataset/TEMP-temporal-dataset-group/INTERACTIONS",
  "ResponseMetadata": {
    "RequestId": "7443253c-04bf-48f8-aada-c04be524e487",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Fri, 18 Jan 2019 18:36:23 GMT",
      "x-amzn-requestid": "7443253c-04bf-48f8-aada-c04be524e487",
      "content-length": "108",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


# Prepare, Create, and Wait for Dataset Import Job

In [20]:
s3 = boto3.client("s3")

policy = {
    "Version": "2012-10-17",
    "Id": "PersonalizeS3BucketAccessPolicy",
    "Statement": [
        {
            "Sid": "PersonalizeS3BucketAccessPolicy",
            "Effect": "Allow",
            "Principal": {
                "Service": "personalize.amazonaws.com"
            },
            "Action": [
                "s3:GetObject",
                "s3:ListBucket"
            ],
            "Resource": [
                "arn:aws:s3:::{}".format(bucket),
                "arn:aws:s3:::{}/*".format(bucket)
            ]
        }
    ]
}

s3.put_bucket_policy(Bucket=bucket, Policy=json.dumps(policy));

### Create S3 Read-Only Access Role

In [24]:
from botocore.exceptions import ClientError

In [28]:
iam = boto3.client("iam")

role_name = "PersonalizeS3Role"
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [
        {
          "Effect": "Allow",
          "Principal": {
            "Service": "personalize.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
        }
    ]
}
try:
    create_role_response = iam.create_role(
        RoleName = role_name,
        AssumeRolePolicyDocument = json.dumps(assume_role_policy_document)
    );

    iam.attach_role_policy(
        RoleName = role_name,
        PolicyArn = "arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess"
    );

    role_arn = create_role_response["Role"]["Arn"]
except ClientError as e:
    if e.response['Error']['Code'] == 'EntityAlreadyExists':
        role_arn = iam.get_role(RoleName=role_name)['Role']['Arn']
    else:
        raise
print(role_arn)

arn:aws:iam::261294318658:role/PersonalizeS3Role


In [29]:
create_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "DEMO-temporal-dataset-import-job",
    datasetArn = dataset_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket, filename)
    },
    roleArn = role_arn
)

dataset_import_job_arn = create_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_dataset_import_job_response, indent=2))

{
  "datasetImportJobArn": "arn:aws:personalize:us-east-1:261294318658:dataset-import-job/TEMP-temporal-dataset-import-job",
  "ResponseMetadata": {
    "RequestId": "e6018a05-66ee-4e76-89b3-f81ac0801985",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Fri, 18 Jan 2019 19:14:48 GMT",
      "x-amzn-requestid": "e6018a05-66ee-4e76-89b3-f81ac0801985",
      "content-length": "120",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


### Wait for Dataset Import Job and Dataset Import Job Run to Have ACTIVE Status

In [30]:
status = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_import_job_response = personalize.describe_dataset_import_job(
        datasetImportJobArn = dataset_import_job_arn
    )
    
    dataset_import_job = describe_dataset_import_job_response["datasetImportJob"]
    if "latestDatasetImportJobRun" not in dataset_import_job:
        status = dataset_import_job["status"]
        print("DatasetImportJob: {}".format(status))
    else:
        status = dataset_import_job["latestDatasetImportJobRun"]["status"]
        print("LatestDatasetImportJobRun: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

DatasetImportJob: CREATE PENDING
LatestDatasetImportJobRun: CREATE IN_PROGRESS
LatestDatasetImportJobRun: CREATE IN_PROGRESS
LatestDatasetImportJobRun: CREATE IN_PROGRESS
LatestDatasetImportJobRun: CREATE IN_PROGRESS
LatestDatasetImportJobRun: CREATE IN_PROGRESS
LatestDatasetImportJobRun: CREATE IN_PROGRESS
LatestDatasetImportJobRun: CREATE IN_PROGRESS
LatestDatasetImportJobRun: CREATE IN_PROGRESS
LatestDatasetImportJobRun: CREATE IN_PROGRESS
LatestDatasetImportJobRun: CREATE IN_PROGRESS
LatestDatasetImportJobRun: CREATE IN_PROGRESS
LatestDatasetImportJobRun: CREATE IN_PROGRESS
LatestDatasetImportJobRun: CREATE IN_PROGRESS
LatestDatasetImportJobRun: CREATE IN_PROGRESS
LatestDatasetImportJobRun: CREATE IN_PROGRESS
LatestDatasetImportJobRun: ACTIVE


# Create Solution

In [7]:
recipe_list = [x['recipeArn'] for x in personalize.list_recipes()['recipes']]
recipe_list

['arn:aws:personalize:::recipe/awspersonalizedeepfmmodel',
 'arn:aws:personalize:::recipe/awspersonalizeffnnmodel',
 'arn:aws:personalize:::recipe/awspersonalizehrnnmodel',
 'arn:aws:personalize:::recipe/awspersonalizehrnnmodel-for-coldstart',
 'arn:aws:personalize:::recipe/awspersonalizehrnnmodel-for-metadata',
 'arn:aws:personalize:::recipe/awspersonalizesimsmodel',
 'arn:aws:personalize:::recipe/popularity-baseline',
 'arn:aws:personalize:::recipe/search-personalization']

There are many recipes for different scenarios. In this example, we only have interactions data, so we will choose one from the basic recipes.

| Feasible? | Recipe | Description 
|-------- | -------- |:------------
| Y | popularity-baseline | Calculates popularity of items based on count of events against that item in user-item interactions dataset.
| Y | awspersonalizedeepfmmodel | Predicts items a user will interact with. Extends Factorization machine using a deep autoencoder to achieve higher accuracy via nonlinear modeling of user item interactions. 
| Y | awspersonalizeffnnmodel | Predicts items a user will interact with. A feed-forward neural network that utilizes a built-in K-nearest neighbors search to deliver recommendations based on past interactions in user-item interactions dataset.
| Y | awspersonalizehrnnmodel | Predicts items a user will interact with. A hierarchical recurrent neural network which can model the temporal order of user-item interactions.
| N - requires meta data | awspersonalizehrnnmodel-for-metadata | Predicts items a user will interact with. HRNN with additional features derived from contextual (user-item interaction metadata), user medata (user dataset) and item metadata (item dataset)
| N - for bandits and requires meta data | awspersonalizehrnnmodel-for-coldstart | Predicts items a user will interact with. HRNN-metadata with with personalized exploration of new items.
| N - for item-based queries | awspersonalizesimsmodel | Computes items similar to a given item based on co-occurrence of item in same user history in user-item interaction dataset
| N - for reranking a short list | search-personalization | Reranks a list of items for a user. Trains on user-item interactions dataset. 


We (or autoML) can run all of these basic recipes and choose the best-performing model from internal metrics. We recommend comparisons, especially with popularity-baseline, to see the lifts in metrics via personalization. However, in this demo, we will pick one recipe - awspersonalizehrnnmodel, to focus on external evaluations.

In [31]:
recipe_arn = "arn:aws:personalize:::recipe/awspersonalizehrnnmodel"

arn:aws:personalize:::recipe/awspersonalizehrnnmodel


In [32]:
create_solution_response = personalize.create_solution(
    name = "DEMO-temporal-solution",
    datasetGroupArn = dataset_group_arn,
    recipeArn = recipe_arn,
    minProvisionedTPS = 1,
)

solution_arn = create_solution_response['solutionArn']
print(json.dumps(create_solution_response, indent=2))

{
  "solutionArn": "arn:aws:personalize:us-east-1:261294318658:solution/TEMP-temporal-solution",
  "ResponseMetadata": {
    "RequestId": "524bfd66-6551-4ec2-9dc3-87fa0276d65f",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Fri, 18 Jan 2019 19:30:54 GMT",
      "x-amzn-requestid": "524bfd66-6551-4ec2-9dc3-87fa0276d65f",
      "content-length": "92",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


### Wait for Solution to Have ACTIVE Status

In [33]:
status = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_solution_response = personalize.describe_solution(
        solutionArn = solution_arn
    )
    status = describe_solution_response["solution"]["status"]
    print("Solution: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

Solution: CREATE PENDING
Solution: CREATE IN_PROGRESS
Solution: CREATE IN_PROGRESS
Solution: CREATE IN_PROGRESS
Solution: CREATE IN_PROGRESS
Solution: CREATE IN_PROGRESS
Solution: CREATE IN_PROGRESS
Solution: CREATE IN_PROGRESS
Solution: CREATE IN_PROGRESS
Solution: CREATE IN_PROGRESS
Solution: CREATE IN_PROGRESS
Solution: CREATE IN_PROGRESS
Solution: CREATE IN_PROGRESS
Solution: CREATE IN_PROGRESS
Solution: CREATE IN_PROGRESS
Solution: CREATE IN_PROGRESS
Solution: CREATE IN_PROGRESS
Solution: CREATE IN_PROGRESS
Solution: CREATE IN_PROGRESS
Solution: CREATE IN_PROGRESS
Solution: CREATE IN_PROGRESS
Solution: CREATE IN_PROGRESS
Solution: CREATE IN_PROGRESS
Solution: CREATE IN_PROGRESS
Solution: CREATE IN_PROGRESS
Solution: CREATE IN_PROGRESS
Solution: CREATE IN_PROGRESS
Solution: CREATE IN_PROGRESS
Solution: CREATE IN_PROGRESS
Solution: CREATE IN_PROGRESS
Solution: CREATE IN_PROGRESS
Solution: CREATE IN_PROGRESS
Solution: CREATE IN_PROGRESS
Solution: CREATE IN_PROGRESS
Solution: CREATE I

### Get Metrics of Solution

In [34]:
get_metrics_response = personalize.get_metrics(
    solutionArn = solution_arn
)

print(json.dumps(get_metrics_response, indent=2))

{
  "metrics": {
    "arn:aws:personalize:us-east-1:261294318658:model/awspersonalizehrnnmodel-4749a326": {
      "_num_evaluation_users": 597.0,
      "_num_unique_items": 3685.0,
      "_user_history_length_10_pct_quantile": 23.0,
      "_user_history_length_50_pct_quantile": 80.0,
      "_user_history_length_90_pct_quantile": 331.9999999999998,
      "_user_history_length_mean": 138.1641541038526,
      "coverage": 0.5443690637720489,
      "mean_reciprocal_rank": 0.09631309678553063,
      "normalized_discounted_cumulative_gain_at_10": 0.1383955191298834,
      "normalized_discounted_cumulative_gain_at_25": 0.17803799204057924,
      "normalized_discounted_cumulative_gain_at_5": 0.11308015473561624,
      "precision_at_10": 0.02194304857621441,
      "precision_at_25": 0.015209380234505866,
      "precision_at_5": 0.029145728643216084
    }
  },
  "ResponseMetadata": {
    "RequestId": "02067381-b661-4690-bb38-2e2761ad68ab",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "co

# Create and Wait for Campaign

In [35]:
create_campaign_response = personalize.create_campaign(
    name = "DEMO-temporal-campaign",
    solutionArn = solution_arn,
    updateMode = "MANUAL"
)

campaign_arn = create_campaign_response['campaignArn']
print(json.dumps(create_campaign_response, indent=2))

{
  "campaignArn": "arn:aws:personalize:us-east-1:261294318658:campaign/TEMP-temporal-campaign",
  "ResponseMetadata": {
    "RequestId": "2e65f038-2355-4a38-9dce-11bd79587dcb",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Fri, 18 Jan 2019 20:25:02 GMT",
      "x-amzn-requestid": "2e65f038-2355-4a38-9dce-11bd79587dcb",
      "content-length": "92",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


### Wait for Campaign to Have ACTIVE Status

In [36]:
status = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_campaign_response = personalize.describe_campaign(
        campaignArn = campaign_arn
    )
    status = describe_campaign_response["campaign"]["status"]
    print("Campaign: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

Campaign: CREATE PENDING
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: ACTIVE


# Evaluate using external metrics

An explanation of the evaluation metrics are provided at https://docs.aws.amazon.com/personalize/latest/dg/working-with-training-metrics.html

For example, suppose we recommend four items and two of them are relevant, $r=[0,1,0,1]$. In this case, the metrics are:

|Name	|Example	|Explanation
|:------|:----------|:----------
|Precision@K	|$\frac{2}{4} = 0.5$	|Total relevant items divided by total recommended items.
|Mean reciprocal ranks (MRR@K)	|${\rm mean}(\frac{1}{2} + \frac{1}{4}) = 0.375$	|Considers positional effects by computing the mean of the inverse positions of all relevant items.
|Normalized discounted cumulative gains (NDCG@K)	|$\frac{\frac{1}{\log(1 + 2)} + \frac{1}{\log(1 + 4)}}{\frac{1}{\log(1 + 1)} + \frac{1}{\log(1 + 2)}} = 0.65$	|Considers positional effects by applying inverse logarithmic weights based on the positions of relevant items, normalized by the largest possible scores from ideal recommendations.
|Average precision (AP@K)	|${\rm mean}(\frac{1}{2} + \frac{2}{4}) = 0.5$	|Average precision@K where K is the position of every relevant item.

These metrics are different from the internal metrics in two aspects:
* They are evaluated at different times, which may imply different click rates. We recommend to always keep the evaluations in the same time periods to avoid temporal drifts.
* The example external evaluations may hold out and consider multiple items as ground truth, while the internal evaluations only hold out the last item in each user-history as the ground truth. There is no absolute preference as to how many items should be held out; we recommend designing the evaluation methods that are similar to the actual use case.

In [123]:
from tqdm import tqdm_notebook
import numpy as np
from metrics import mean_reciprocal_rank, ndcg_at_k, precision_at_k

In [62]:
relevance = []
for user_id, true_items in tqdm_notebook(holdout.groupby('USER_ID').ITEM_ID):
    rec_response = personalize_runtime.get_recommendations(
        campaignArn = campaign_arn,
        userId = str(user_id)
    )
    rec_items = [int(x['itemId']) for x in rec_response['itemList']]
    relevance.append([int(x in true_items.values) for x in rec_items])

In [128]:
print('mean_reciprocal_rank', np.mean([mean_reciprocal_rank(r) for r in relevance]))
print('precision_at_5', np.mean([precision_at_k(r, 5) for r in relevance]))
print('precision_at_10', np.mean([precision_at_k(r, 10) for r in relevance]))
print('precision_at_25', np.mean([precision_at_k(r, 25) for r in relevance]))
print('normalized_discounted_cumulative_gain_at_5', np.mean([ndcg_at_k(r, 5) for r in relevance]))
print('normalized_discounted_cumulative_gain_at_10', np.mean([ndcg_at_k(r, 10) for r in relevance]))
print('normalized_discounted_cumulative_gain_at_25', np.mean([ndcg_at_k(r, 25) for r in relevance]))

mean_reciprocal_rank 0.14581661677556126
precision_at_5 0.11615894039735097
precision_at_10 0.10548013245033112
precision_at_25 0.0865430463576159
normalized_discounted_cumulative_gain_at_5 0.17839946367779197
normalized_discounted_cumulative_gain_at_10 0.2395355109194227
normalized_discounted_cumulative_gain_at_25 0.35113161206085886
